# Example notebook showcasing investigation of a small outbreak of Vancomycin Resistant Enterococcus (VRE) in patient samples

Data has previously gone through QC and anlysis via the CZID pipeline https://chanzuckerberg.zendesk.com/hc/en-us/articles/360050326971-Guide-Data-Analysis 

## Preliminary steps & getting files ready for analysis

In [1]:
import os
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### This is where the directory is mounted at (/data)

In [2]:
import seaborn as sns


In [3]:
pwd

'/home/randall.white/spid_singularity'

In [4]:
cd ../

/home/randall.white


In [5]:
ls

Desktop/
Documents/
Downloads/
Manifest.toml
Music/
ParaView-5.12.0-RC1-MPI-Linux-Python3.10-x86_64/
ParaView-5.12.0-RC1-MPI-Linux-Python3.10-x86_64.tar.gz
Pictures/
Project.toml
Public/
R/
Templates/
Untitled.ipynb
Untitled1.ipynb
Untitled10.ipynb
Untitled11.ipynb
Untitled12.ipynb
Untitled13.ipynb
Untitled14.ipynb
Untitled15.ipynb
Untitled16.ipynb
Untitled17.ipynb
Untitled2.ipynb
Untitled3.ipynb
Untitled4.ipynb
Untitled5.ipynb
Untitled6.ipynb
Untitled7.ipynb
Untitled8.ipynb
Untitled9.ipynb
Videos/
aocc-compiler-4.1.0/
aocc-compiler-4.1.0.tar
aocc-compiler-4.1.0.tar.1
aocl-linux-aocc-4.1.0/
aocl-linux-aocc-4.1.0.tar.gz
config.py
cpuid/
cryosparc-v2/
downloadTestData.sh
griznog_repo/
gui_automation.sif*
hpc/
id_ecdsa
id_ecdsa.pub
job_submission1.sh
job_submission1.sh.save
job_submission2.sh
my_macbook_keys.tar.xz
nodeList.txt
ondemand/
packageList.txt
pkgs/
raxml-ng_v1.2.1_linux_x86_64.zip
raxml-ng_v1.2.1_linux_x86_64_MPI.zip
rc_apps/
rc_repos/
rclone_test.sh
send_msgto_webhook_11750048

In [6]:
cd spidtest

/hpc/scratch/group.data.science/eric_temp/spid/RR192/spidtest


In [7]:
cd ..

/hpc/scratch/group.data.science/eric_temp/spid/RR192


In [8]:
!head CP011306.1.fasta

>CP011306.1 Stenotrophomonas maltophilia strain ISMMS2R, complete genome
AGGCGGGCGCTCGGTGGTAGTCTGAGCCATCCATTTCCCCCCTTTCCGGCCTGTGTTTGGGCGCCTTGTC
GCGTCCATGCCCCGGCCACCGGACGATTATTGCTGATGGATGCTTGGTCCCGTAGTCTCGAGCGCCTCGA
AGCGGAGTTCCCGCCGGAAGACGTTCATACCTGGTTGAAGCCGCTGCAGGCCGATCTGCGCGTGGACAGC
CTGGTGCTGTATGCACCGAATGCCTTCATCGTCGACCAGGTCCGCGAGCTGTATCTGGCCCGGATCCGCG
AACTGCTGGCTCATTTCGCCGGCTTCAGCGACGTTTTTCTTGAAATCGGCTCGCGCCCGCGCCCTGTGGA
GGCGCAGAACACGCCGGTTTCCACGTCATCGGCGCATGTGTCCAGCGAACCGCAGGTGCCGTTCGCCGGC
AACCTGGACAATCACTACACGTTCGCCAACTTCGTGGAAGGCCGCAGCAACCAGCTGGGCCTGGCCGCCG
CCTTCCAGGCAGCGCAGAAGCCGGGCGACCGCGCGCACAACCCACTGCTGCTGTACGGAGGCACCGGCCT
GGGCAAGACCCACCTGATGTTCGCCGCCGGCAACGCCATGCGCCAGGCCAATCCGGGCGCGAAGGTACTG


In [9]:
sample_fastas = !(ls RR192*)
sample_fastas

['RR192_01_T13825_DNA_Seq111_S1_430035_reads_nh_Stenotroph.fasta',
 'RR192_02_F3320_DNA_Seq111_S2_430036_reads_nh_Stenotroph.fasta',
 'RR192_03_X49465_DNA_Seq111_S3_430037_reads_nh_Stenotroph.fasta',
 'RR192_04_T18653_DNA_Seq111_S4_430038_reads_nh_Stenotroph.fasta',
 'RR192_05_F19310_DNA_Seq111_S5_430039_reads_nh_Stenotroph.fasta',
 'RR192_06_S12546_DNA_Seq111_S6_430040_reads_nh_Stenotroph.fasta',
 'RR192_07_W54616_DNA_Seq115_S10_459587_reads_nh_Stenotroph.fasta']

In [10]:
names = []
for file in sample_fastas:
    sample_name_only = '_'.join(file.split('_')[:-4])
    names.append(sample_name_only)
names

['RR192_01_T13825_DNA_Seq111_S1',
 'RR192_02_F3320_DNA_Seq111_S2',
 'RR192_03_X49465_DNA_Seq111_S3',
 'RR192_04_T18653_DNA_Seq111_S4',
 'RR192_05_F19310_DNA_Seq111_S5',
 'RR192_06_S12546_DNA_Seq111_S6',
 'RR192_07_W54616_DNA_Seq115_S10']

In [11]:
sample_names_paths = dict(zip(names, sample_fastas))
sample_names_paths

{'RR192_01_T13825_DNA_Seq111_S1': 'RR192_01_T13825_DNA_Seq111_S1_430035_reads_nh_Stenotroph.fasta',
 'RR192_02_F3320_DNA_Seq111_S2': 'RR192_02_F3320_DNA_Seq111_S2_430036_reads_nh_Stenotroph.fasta',
 'RR192_03_X49465_DNA_Seq111_S3': 'RR192_03_X49465_DNA_Seq111_S3_430037_reads_nh_Stenotroph.fasta',
 'RR192_04_T18653_DNA_Seq111_S4': 'RR192_04_T18653_DNA_Seq111_S4_430038_reads_nh_Stenotroph.fasta',
 'RR192_05_F19310_DNA_Seq111_S5': 'RR192_05_F19310_DNA_Seq111_S5_430039_reads_nh_Stenotroph.fasta',
 'RR192_06_S12546_DNA_Seq111_S6': 'RR192_06_S12546_DNA_Seq111_S6_430040_reads_nh_Stenotroph.fasta',
 'RR192_07_W54616_DNA_Seq115_S10': 'RR192_07_W54616_DNA_Seq115_S10_459587_reads_nh_Stenotroph.fasta'}

### Running the first step (short_read_alignments)

In [20]:
mkdir -p short_read_alignments

In [12]:
ls -ltr

total 4115640
-rw-r--r-- 1 eric.waltari nogroup   4574222 Nov 21 10:57 CP011306.1.fasta
-rw-r--r-- 1 eric.waltari nogroup   4545227 Nov 21 10:57 GCF_900475405.1.fasta
-rw-r--r-- 1 eric.waltari nogroup 452984832 Nov 21 10:57 RR192_05_F19310_DNA_Seq111_S5_430039_reads_nh_Stenotroph.fasta
-rw-r--r-- 1 eric.waltari nogroup 618286360 Nov 21 10:58 RR192_01_T13825_DNA_Seq111_S1_430035_reads_nh_Stenotroph.fasta
-rw-r--r-- 1 eric.waltari nogroup 625518589 Nov 21 10:58 RR192_03_X49465_DNA_Seq111_S3_430037_reads_nh_Stenotroph.fasta
-rw-r--r-- 1 eric.waltari nogroup 613593802 Nov 21 10:58 RR192_02_F3320_DNA_Seq111_S2_430036_reads_nh_Stenotroph.fasta
-rw-r--r-- 1 eric.waltari nogroup 594285914 Nov 21 10:58 RR192_07_W54616_DNA_Seq115_S10_459587_reads_nh_Stenotroph.fasta
-rw-r--r-- 1 eric.waltari nogroup 615869296 Nov 21 10:58 RR192_04_T18653_DNA_Seq111_S4_430038_reads_nh_Stenotroph.fasta
-rw-r--r-- 1 eric.waltari nogroup 594068362 Nov 21 10:59 RR192_06_S12546_DNA_Seq111_S6_430040_reads_nh_Stenotroph

**note the reference we're using here is called LR135331.fasta** This reference is selected based on the CZID pipeline.

In [13]:
align_short_read_commands = [f"'spid.jl align_short_reads --threads 16 short_read_alignments/{names} CP096540.1.fasta {fasta_files}'" for names, fasta_files in sample_names_paths.items()]


In [14]:
align_short_read_commands

["'spid.jl align_short_reads --threads 16 short_read_alignments/RR192_01_T13825_DNA_Seq111_S1 CP096540.1.fasta RR192_01_T13825_DNA_Seq111_S1_430035_reads_nh_Stenotroph.fasta'",
 "'spid.jl align_short_reads --threads 16 short_read_alignments/RR192_02_F3320_DNA_Seq111_S2 CP096540.1.fasta RR192_02_F3320_DNA_Seq111_S2_430036_reads_nh_Stenotroph.fasta'",
 "'spid.jl align_short_reads --threads 16 short_read_alignments/RR192_03_X49465_DNA_Seq111_S3 CP096540.1.fasta RR192_03_X49465_DNA_Seq111_S3_430037_reads_nh_Stenotroph.fasta'",
 "'spid.jl align_short_reads --threads 16 short_read_alignments/RR192_04_T18653_DNA_Seq111_S4 CP096540.1.fasta RR192_04_T18653_DNA_Seq111_S4_430038_reads_nh_Stenotroph.fasta'",
 "'spid.jl align_short_reads --threads 16 short_read_alignments/RR192_05_F19310_DNA_Seq111_S5 CP096540.1.fasta RR192_05_F19310_DNA_Seq111_S5_430039_reads_nh_Stenotroph.fasta'",
 "'spid.jl align_short_reads --threads 16 short_read_alignments/RR192_06_S12546_DNA_Seq111_S6 CP096540.1.fasta RR192_

In [15]:
os.system(f" parallel ::: {' '.join(align_short_read_commands)}")

sh: 1: parallel: not found


32512

In [20]:
""'spid.jl align_short_reads --threads 16 short_read_alignments/RR192_01_T13825_DNA_Seq111_S1 CP096540.1.fasta RR192_01_T13825_DNA_Seq111_S1_430035_reads_nh_Stenotroph.fasta'""

'spid.jl align_short_reads --threads 16 short_read_alignments/RR192_01_T13825_DNA_Seq111_S1 CP096540.1.fasta RR192_01_T13825_DNA_Seq111_S1_430035_reads_nh_Stenotroph.fasta'

In [19]:
ls -all short_read_alignments/

total 8
drwxr-sr-x 2 eric.waltari nogroup 4096 Feb 27 11:39 ./
drwxr-sr-x 5 eric.waltari nogroup 4096 Feb 27 11:39 ../


## align_assembly step if you have other references

Command format: **spid.jl align_assembly {output filename} {orig_reference} {assembly_reference}**

In [ ]:
mkdir -p align_assembly

In [ ]:
#run align_assembly step with our "original" reference + ref 2 and "original" ref + ref 3
#Here, CP019992 and CP025425 are two other references identified in CZID and downloaded from NCBI. 

align_assembly_commands = ["'spid.jl align_assembly align_assembly/MRSAref2 CP096540.1.fasta ER01836.3.fasta'"]

In [ ]:
align_assembly_commands

In [ ]:
os.system(f" parallel ::: {' '.join(align_assembly_commands)}")

In [ ]:
ls -ltr align_assembly/

## Last step - merge_alignments

**now, to do merge_alignments, we have to combine all the fastas we want to merge together, 
from short_read_alignments folder and the align_assembly folder**

In [ ]:
!mkdir -p files_to_do_merge_alignments

In [ ]:
ls

In [ ]:
!cp align_assembly/M*.fa files_to_do_merge_alignments/

In [ ]:
ls files_to_do_merge_alignments/

In [ ]:
!cp short_read_alignments/*.fa files_to_do_merge_alignments/

In [ ]:
ls files_to_do_merge_alignments/

In [ ]:
files_for_merge = !(ls files_to_do_merge_alignments/*.fa)
files_for_merge

In [ ]:
{' '.join([f'{fasta_files}' for fasta_files in files_for_merge])}


**Here, "vre_samples" is the name of subsequent output files, including the snp matrix**

In [ ]:
os.system(f" spid.jl merge_alignments RR108_Sept2023 CP096540.1.fasta {' '.join([f'{fasta_files}' for fasta_files in files_for_merge])}")


In [ ]:
ls -ltr

## Looking at outputs

In [ ]:
!head RR108_Sept2023.fa.pairwise_diffs.csv

In [ ]:
pairwise_diffs = pd.read_csv('RR108_Sept2023.fa.pairwise_diffs.csv')
pairwise_diffs

In [ ]:
plt.hist(pairwise_diffs['SharedGenomeLen'], bins=25)

**Adding a column that is snps/kb (divide numdiffs/sharedgenomelen then *1000)**

In [ ]:
pairwise_diffs['snps/kb'] = pairwise_diffs['NumDiffs']/pairwise_diffs['SharedGenomeLen'] * 1000
pairwise_diffs

In [ ]:
## April 2023 added code to identify samples with very short alignments...BEST TO REMOVE THESE AND RE-RUN!!
## R equivalent (translated via chatGPT)
#dataframe2 <- dataframe %>%
#  group_by(name1) %>%
#  summarize(length = mean(SharedGenomeLen))
# followed by
#pairwise_diffs_len_short <- pairwise_diffs_len %>%
#  filter(meanlen < (mean(pairwise_diffs_len$meanlen) * 0.75))
#pairwise_diffs_len_short$Contig1

pairwise_diffs2 = pairwise_diffs.groupby('Contig1').agg({'SharedGenomeLen': 'mean'}).reset_index()
#pairwise_diffs2 = pairwise_diffs2.rename(columns={'SharedGenomeLen': 'meanlength'})
#This code groups the DataFrame dataframe by the name1 column, calculates the mean of the SharedGenomeLen column for each group,
# and returns a new DataFrame dataframe2 with the resulting summary.
#The reset_index() function is used to convert the grouped DataFrame into a regular DataFrame, 
# and the rename() function is used to rename the resulting column SharedGenomeLen to length.

pairwise_diffs_short = pairwise_diffs2[pairwise_diffs2['SharedGenomeLen'] < (pairwise_diffs['SharedGenomeLen'].mean() * 0.75)]
pairwise_diffs_short
## AGAIN RECOMMENDATION IS TO REMOVE THESE SAMPLES AND RE-RUN BEFORE MOVING TO R SCRIPT

In [ ]:
plt.hist(pairwise_diffs['snps/kb'],  bins=50, color='g')

In [ ]:
 pivoted_snps = pairwise_diffs.pivot(index="Contig1", columns="Contig2", values="snps/kb")
pivoted_snps

### Plotting heatmap of the snp differences between samples ###

In [ ]:
sns.heatmap(pivoted_snps, cmap='viridis')

In [ ]:
heatmap = sns.heatmap(pivoted_snps, fmt=".2f", cmap='viridis', annot=True, linewidths=0.5)

In [ ]:
fig = heatmap.get_figure()
fig.savefig('snps_per_kb_vre_samples.png', dpi=300, bbox_inches='tight', pad_inches=0.5)